In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

In [4]:
import configparser, networkx as nx, itertools, math, json, sys, time, timeit, numpy, pdb, jsbeautifier, re, spacy
from tqdm import tqdm


### Concept Grounding
Extract and match concepts from Ai2Thor objects list to ConceptNet concepts.

In [2]:
from grounding.grounding_concepts import GroundConcepts

In [15]:
config_path = Path("/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/paths.cfg")
ground = GroundConcepts(config_path)

In [22]:
ground.process('/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/objects.txt', )

Added rooms


grounding: 100%|██████████| 87/87 [09:37<00:00,  6.64s/it]


Saved to /Users/odoemoo1/projects/visual-navigation/KagNet/datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json


We can skip pruning step as we're not dealing with sentences. 

### Generate Conceptnet graph

Run below in cell:
```
!cd ./pathfinder/`
!python graph_construction.py`
!cd ..
```

Outputs networkX graph of conceptnet.

### Path finding
Find a path between the concepts (i.e. objects).

Re-factored pathfinder.py to create an object. 

In [3]:
from pathfinder.pathfinder import PathFinder

In [8]:
pathfinder = PathFinder( '/Users/odoemoo1/projects/visual-navigation/KagNet/pathfinder/paths.cfg')
fn = Path('./datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json')

loading conceptnet graph...
loading conceptnet graph...done


In [12]:
pathfinder.process(fn, nhops=1)

100%|██████████| 87/87 [00:00<00:00, 196.78it/s]


Found paths written to datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json.1hops_paths.json


In [46]:
from dgl import DGLGraph

Using backend: pytorch


In [47]:
dg = DGLGraph()
dg.from_networkx(cpnet_simple)

/Users/odoemoo1/anaconda3/envs/kagnet_test/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)


In [49]:
cpnet_simple.number_of_nodes()

669941

In [61]:
dg.subgraph([10, 20, 500]).adjacency_matrix()

tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(3, 3), nnz=0, layout=torch.sparse_coo)

TODO:
    - can you get a subgraph with just the nodes of AI2thor objects as concepts, without pathfinding? 